In [1]:
import torch
import torch.nn as nn
import torch.utils
import torchvision
import torchvision.transforms as transforms
from PIL import Image
import numpy as np
import torch.nn.functional as F

MODEL

In [2]:
classes=('plane','car','bird','cat',
         'deer','dog','frog','horse',
         'ship','truck')

class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet,self).__init__()
        self.conv1=nn.Conv2d(3,6,5)
        self.pool=nn.MaxPool2d(2,2)
        self.conv2=nn.Conv2d(6,16,5)
        self.fc1=nn.Linear(16*5*5,120)
        self.fc2=nn.Linear(120,84)
        self.fc3=nn.Linear(84,10) # 120 and 84 can be changed

    def forward(self,x):
        x=self.pool(F.relu(self.conv1(x)))
        x=self.pool(F.relu(self.conv2(x)))
        x=x.view(-1,16*5*5)
        x = self.fc1(x)
        x = F.relu(x)  
        x = F.relu(self.fc2(x))
        x=self.fc3(x)
        return x

TRAIN A MODEL - change parameters if needed

In [ ]:

num_epochs=400
batch_size=4
learning_rate=0.001

transform=transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))])

#CIFAR10
train_dataset =torchvision.datasets.CIFAR10(root='./data',train=True,download=True,transform=transform)
test_dataset =torchvision.datasets.CIFAR10(root='./data',train=False,download=True,transform=transform)

train_loader=torch.utils.data.DataLoader(train_dataset,batch_size=batch_size,shuffle=True)
test_loader=torch.utils.data.DataLoader(test_dataset,batch_size=batch_size,shuffle=False)

    
model=ConvNet()

#loss and optimizer
criterion=nn.CrossEntropyLoss()
optimizer=torch.optim.SGD(model.parameters(),lr=learning_rate)


#training loop

n_total_steps=len(train_loader)
for epoch in range(num_epochs):
    for i, (images,labels) in enumerate(train_loader):
        #forward 
        outputs=model(images)
        loss = criterion(outputs,labels)
        
        # backward
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1)%2000==0:
            print(f'epoch {epoch+1}/{num_epochs}, step {i+1}/{n_total_steps},loss = {loss.item():.4f}')

# test
with torch.no_grad():
    n_correct=0
    n_samples=0
    n_class_correct=np.zeros(10)
    n_class_samples=np.zeros(10)
    for images, labels in test_loader:

        outputs=model(images)
        #value, index 
        _,predicted=torch.max(outputs,1)
        
        n_samples+=labels.size(0)
        n_correct+=(predicted==labels).sum().item()
        
        for i in range(batch_size):
            label=labels[i]
            pred=predicted[i]
            if(label==pred):
                n_class_correct[label]+=1
            n_class_samples[label]+=1
            
            
        
    acc = 100.0*n_correct/n_samples
    print(f'accuracy = {acc} %')
    
    for i in range(10):
        acc= 100.0*n_class_correct[i]/n_class_samples[i]
        print(f'Accuracy of {classes[i]}: {acc} %')
        
# Save the model
torch.save(model.state_dict(), 'saved_model.pth')

MAKE A PREDICTION WITH A SAVED MODEL

In [3]:
model=ConvNet()
model.load_state_dict(torch.load('saved_model.pth'))
model.eval()

img="dog.jpg"# add more photos if needed
img2="cat.jpg"
img3="ship.jpg"
input_img = Image.open(img)
input_img2 = Image.open(img2)
input_img3 = Image.open(img3)

# Apply transformations to the input image
transform = transforms.Compose([
    transforms.Resize((32, 32)),  # Resize the image to match the input size of the model
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

input_tensor = transform(input_img).unsqueeze(0)
input_tensor2 = transform(input_img2).unsqueeze(0)
input_tensor3 = transform(input_img3).unsqueeze(0)

# Make predictions
with torch.no_grad():
    output = model(input_tensor)
    _, predicted = torch.max(output, 1)
    output2 = model(input_tensor2)
    _, predicted2 = torch.max(output2, 1)
    output3 = model(input_tensor3)
    _, predicted3 = torch.max(output3, 1)
    
print("Predicted class, based on "+img+":", classes[predicted.item()])
print("Predicted class, based on "+img2+":", classes[predicted2.item()])
print("Predicted class, based on "+img3+":", classes[predicted3.item()])

Predicted class, based on dog.jpg: dog
Predicted class, based on cat.jpg: cat
Predicted class, based on ship.jpg: ship
